In [2]:
import pandas as pd
import numpy as np

train_df = pd.read_json('Untitled Folder/train_whats_cooking.json')
test_df = pd.read_json('Untitled Folder/test_whats_cooking.json')

train_df['seperated_ingredients'] = train_df['ingredients'].map(" , ".join)
test_df['seperated_ingredients'] = test_df['ingredients'].map(",".join)


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer#CountVectorizer

vect = TfidfVectorizer(binary=True).fit(train_df['seperated_ingredients'].values)
X_train_vectorized = vect.transform(train_df['seperated_ingredients'].values)


In [4]:
X_train_vectorized = X_train_vectorized.astype('float16')

In [5]:
Result_transformed = vect.transform(test_df['seperated_ingredients'].values)
Result_transformed = Result_transformed.astype('float16')

In [6]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_transformed = encoder.fit_transform(train_df.cuisine)

In [7]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_vectorized, y_transformed , random_state = 0)

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train , y_train)

C:\Users\Husnain\Downloads\ANACONDA\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [8]:
clf.score(X_test, y_test)

In [10]:
y_predicted = clf.predict(Result_transformed)
y_predicted_final = encoder.inverse_transform(y_predicted)
answer = pd.DataFrame({'cuisine' : y_predicted_final , 'id' : test_df.id })
answer = answer[[ 'id' , 'cuisine']]
answer.head()

,id,cuisine
0,18009,british
1,28583,southern_us
2,41580,italian
3,29752,cajun_creole
4,35687,italian


In [10]:
answer.to_csv('kaggle_cuisines6.csv' , encoding = 'utf-8' , index = False)